In [1]:
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.utils import grid
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.nn import Linear, ReLU, Dropout, BatchNorm1d, CrossEntropyLoss
from torch.optim import Adam
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score
import itertools

In [2]:
# Load the dataset
file_path = 'C:/Users/Lenovo/AutoCuroAssignment/tic_tac_toe_records_minimax_Preprocessed.csv'
data = pd.read_csv(file_path)

# Extract inputs and targets
input_graphs = data['Input Graph'].apply(lambda x: eval(x))  # Directly extract as list
targets = data['decision'] - 1  # Subtract 1 to get target range 0-8

# Create PyTorch Geometric Data objects
data_list = []
for i in range(len(input_graphs)):
    # Create a fully connected graph edge_index for a 3x3 grid
    num_nodes = 9
    edge_index = torch.tensor(list(itertools.permutations(range(num_nodes), 2)), dtype=torch.long).t().contiguous()
    
    # Flatten the 3x3 grid into a 9-element feature vector
    x = torch.tensor([val for row in input_graphs[i] for val in row], dtype=torch.float).view(-1, 1)  # Convert to tensor
    y = torch.tensor(targets[i], dtype=torch.long)
    data_list.append(Data(x=x, edge_index=edge_index, y=y))

In [3]:
# Create a DataLoader
batch_size = 64
loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)

C:\Users\Lenovo\anaconda3\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [4]:
class ComplexGNNModel(torch.nn.Module):
    def __init__(self):
        super(ComplexGNNModel, self).__init__()
        self.conv1 = GCNConv(1, 32)
        self.conv2 = GCNConv(32, 64)
        self.conv3 = GCNConv(64, 128)
        self.conv4 = GCNConv(128, 256)

        self.batch_norm1 = BatchNorm1d(32)
        self.batch_norm2 = BatchNorm1d(64)
        self.batch_norm3 = BatchNorm1d(128)
        self.batch_norm4 = BatchNorm1d(256)

        self.fc1 = Linear(256, 512)
        self.fc2 = Linear(512, 256)
        self.fc3 = Linear(256, 9)  # 9 possible moves (0-8)
        
        self.relu = ReLU()
        self.dropout = Dropout(p=0.2)  # Dropout rate as specified    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        batch = data.batch  # Get batch tensor

        # Forward pass through GCN layers
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.conv3(x, edge_index)
        x = self.batch_norm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.conv4(x, edge_index)
        x = self.batch_norm4(x)
        x = self.relu(x)
        x = self.dropout(x)

        # Global mean pooling
        x = global_mean_pool(x, batch)

        # Fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)

        return x

In [5]:
# Instantiate the model, loss function, and optimizer
model = ComplexGNNModel()
criterion = CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
#scheduler = StepLR(optimizer, step_size=20, gamma=0.5)

# Split data into training and validation sets
split_ratio = 0.8
split_index = int(len(data_list) * split_ratio)
train_data = data_list[:split_index]
val_data = data_list[split_index:]

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [8]:

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    correct_train = 0
    total_train = 0

    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch)  # Output shape should be [batch_size, 9]
        loss = criterion(out, batch.y)  # Target shape should be [batch_size]
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        _, predicted = torch.max(out, dim=1)
        correct_train += (predicted == batch.y).sum().item()
        total_train += batch.y.size(0)
    
    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = correct_train / total_train

    model.eval()
    total_val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for batch in val_loader:
            out = model(batch)
            loss = criterion(out, batch.y)
            total_val_loss += loss.item()

            _, predicted = torch.max(out, dim=1)
            correct_val += (predicted == batch.y).sum().item()
            total_val += batch.y.size(0)

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = correct_val / total_val

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, '
          f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')


Epoch 1/200, Training Loss: 1.8157, Training Accuracy: 0.3286, Validation Loss: 1.8374, Validation Accuracy: 0.3119
Epoch 2/200, Training Loss: 1.8146, Training Accuracy: 0.3290, Validation Loss: 1.8357, Validation Accuracy: 0.3119
Epoch 3/200, Training Loss: 1.8137, Training Accuracy: 0.3287, Validation Loss: 1.8362, Validation Accuracy: 0.3119


KeyboardInterrupt: 

In [11]:
all_preds

tensor([4, 4, 4,  ..., 4, 4, 4])

In [7]:
# Save the model
torch.save(model.state_dict(), 'C:/Users/Lenovo/AutoCuroAssignment/complex_gnn_model.pth')

# Evaluation on the entire dataset
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for batch in DataLoader(data_list, batch_size=batch_size, shuffle=False):
        out = model(batch)
        preds = torch.argmax(out, dim=1)
        all_preds.append(preds)
        all_targets.append(batch.y)

# Convert lists to tensors
all_preds = torch.cat(all_preds, dim=0)
all_targets = torch.cat(all_targets, dim=0)

# Calculate accuracy
accuracy = accuracy_score(all_targets.cpu(), all_preds.cpu())
print(f'Final Accuracy on the Entire Dataset: {accuracy:.4f}')

Final Accuracy on the Entire Dataset: 0.3253


In [ ]:
#test output
data=[[2, 1, 0], [0, 0, 2], [0, 1, 1]]
num_nodes = 9
edge_index = torch.tensor(list(itertools.permutations(range(num_nodes), 2)), dtype=torch.long).t().contiguous()

# Flatten the 3x3 grid into a 9-element feature vector
x = torch.tensor([val for row in data for val in row], dtype=torch.float).view(-1, 1)  # Convert to tensor

In [3]:
model=ComplexGNNModel()
model.load_state_dict(torch.load('C:/Users/Lenovo/AutoCuroAssignment/complex_gnn_model.pth'))

<All keys matched successfully>

In [18]:
import torch
from torch_geometric.data import Data

# Define the adjacency list for a 3x3 grid
edges = [
    (0, 1), (1, 0), (1, 2), (2, 1),  # Top row
    (0, 3), (3, 0), (1, 4), (4, 1), (2, 5), (5, 2),  # Top row connected to middle row
    (3, 4), (4, 3), (4, 5), (5, 4),  # Middle row
    (3, 6), (6, 3), (4, 7), (7, 4), (5, 8), (8, 5),  # Middle row connected to bottom row
    (6, 7), (7, 6), (7, 8), (8, 7)  # Bottom row
]

# Convert edge list to torch tensor
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Your input matrix
input_matrix = [[1,1, 2], [2, 2, 1], [1,1, 0]]
input_tensor = torch.tensor(input_matrix, dtype=torch.float).view(-1, 1)  # Flatten to [9, 1]

# Create a Data object
data = Data(x=input_tensor, edge_index=edge_index)

# Ensure the model is in evaluation mode
model.eval()

# Make the prediction
with torch.no_grad():
    output = model(data)

# Interpret the output
predicted_move = torch.argmax(output, dim=1).item()

print(f'Predicted move: {predicted_move}')
print(f'Model Output:{output}')

Predicted move: 4
Model Output:tensor([[ 0.1044, -0.3484,  0.2249, -0.2405,  0.4315, -0.2683,  0.1721, -0.3870,
          0.1929]])
